## Imports

In [ ]:
import copy
import gpxpy
from gpxpy.gpx import GPXTrackPoint

import numpy as np
from numpy import ndarray
from sklearn.metrics import DistanceMetric

# import xml.etree.ElementTree as et
from plotly.offline import iplot, plot, init_notebook_mode
import plotly.graph_objs as go

init_notebook_mode()

In [ ]:
%load_ext jupyter_black

## Loading

In [ ]:
with open("activity_data/250729_Morning_Run_orig.gpx", "r") as gpx_file:
    gpx_initial = gpxpy.parse(gpx_file)

gpx = copy.deepcopy(gpx_initial)
points = gpx.tracks[0].segments[0].points
print(f"{len(points) = }")

In [ ]:
def compute_local_correction(point: GPXTrackPoint) -> float:
    """Computes the local correction that makes 1 unit change in lat equal to 1 unit in lon."""
    dist = DistanceMetric.get_metric("haversine")
    epsilon = 0.00001
    point_radians = (np.pi / 180) * np.array([point.latitude, point.longitude])

    ddist_lat = dist.pairwise(np.array([point_radians, point_radians + [epsilon, 0]]))
    ddist_lon = dist.pairwise(np.array([point_radians, point_radians + [0, epsilon]]))
    correction = ddist_lat[0, 1] / ddist_lon[1, 0]

    return correction


def trace_from_points(points: list[GPXTrackPoint], correction_factor: float) -> list[str]:
    trace = go.Scatter(
        x=[point.longitude for point in points],
        y=[point.latitude * correction_factor for point in points],
        text=[f"id: {i}" for i in list(range(len(points)))],
    )

    return trace

## Chart data

### Compute local correction factor

### Chart

In [ ]:
correction_factor = compute_local_correction(points[0])
data = [trace_from_points(points, correction_factor)]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

## Repair data

### with interpolation

In [ ]:
def interpolate_segment(
    points: list[GPXTrackPoint], start_ix: int, end_ix: int
) -> list[GPXTrackPoint]:
    """Interpolates between two points, given by index.  Returns the original
    section for convenience of plotting."""
    orig_segment = copy.deepcopy(points[start_ix : end_ix + 1])

    start_point = np.array(
        [
            points[start_ix].latitude,
            points[start_ix].longitude,
            points[start_ix].elevation,
        ]
    )
    end_point = np.array(
        [points[end_ix].latitude, points[end_ix].longitude, points[end_ix].elevation]
    )

    start_time = points[start_ix].time
    end_time = points[end_ix].time

    dpos = end_point - start_point
    dt = end_time - start_time

    for point in points[start_ix + 1 : end_ix]:
        p = (point.time - start_time) / dt
        new_point = start_point + p * dpos

        point.latitude = round(new_point[0], 7)
        point.longitude = round(new_point[1], 7)
        point.elevation = round(new_point[2], 1)

    return orig_segment

In [ ]:
start_ix = 251
end_ix = 300

In [ ]:
orig_segment = interpolate_segment(points, start_ix, end_ix)

In [ ]:
correction_factor = compute_local_correction(points[0])

data = [
    trace_from_points(points, correction_factor),
    trace_from_points(orig_segment, correction_factor),
]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

### with section removal

In [ ]:
def delete_segment(points: list[GPXTrackPoint], start_ix, end_ix) -> list[GPXTrackPoint]:
    """Deletes a section of bad data from Strava. Strava will interpret this
    as the watch being stopped, and simply not count this interval."""
    orig_segment = copy.deepcopy(points[start_ix : end_ix + 1])
    del points[start_ix : end_ix + 1]

    return orig_segment

In [ ]:
start_ix = 1089
end_ix = 2466

orig_segment = delete_segment(points, start_ix, end_ix)
correction_factor = compute_local_correction(points[0])

data = [
    trace_from_points(points, correction_factor),
    trace_from_points(orig_segment, correction_factor),
]

layout = go.Layout(
    yaxis=dict(scaleanchor="x", scaleratio=1),
    height=600,
)

iplot(go.Figure(data=data, layout=layout))

In [ ]:
len(points)

## Export

In [ ]:
with open("activity_data/250729_Morning_Run_fixed.gpx", "w") as out_file:
    out_file.write(gpx.to_xml())